# Chatbot con **tus archivos** + Gemini + Telegram
By *Ing. Engler Gonzalez*


**Sube todo a la carpeta `/content`**.
El cuaderno ya la usa por defecto.
Flujo:
1) Instalar  
2) Subir archivos  
3) Crear KB  
4) Entrenar  
5) Gemini
6) Telegram.


In [1]:
# ⬇️ 1) Instalación (ejecuta esta celda primero)
%%bash
pip -q install pdfplumber pillow pytesseract speechrecognition pydub python-telegram-bot google-generativeai scikit-learn > /dev/null
if command -v apt >/dev/null 2>&1; then
  apt -qq update >/dev/null && apt -qq install -y tesseract-ocr >/dev/null || true
fi
echo 'Instalación lista ✅'

Instalación lista ✅




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [2]:
# 2) Config de ruta: forzar uso de /content si existe
import os, pathlib
if os.path.isdir('/content'):
    os.chdir('/content')
print('📂 Carpeta de trabajo:', pathlib.Path('.').resolve())
print('📄 Archivos actuales:', os.listdir())

📂 Carpeta de trabajo: /content
📄 Archivos actuales: ['.config', 'sample_data']


## 🔑 Claves / Tokens
- **Gemini (opcional)**: pega tu `GOOGLE_API_KEY`. Si está vacío, usarás TF‑IDF.
- **Telegram**: crea un bot con **@BotFather** y pega el `TELEGRAM_TOKEN`.

In [12]:
import os
GOOGLE_API_KEY = ""  # ej: "AIza..."
TELEGRAM_TOKEN = "8479406629:AAGWLu5Fz6sUUZC74AR4Gc8L0jz5Nm34Dpk"  # ej: "123456:ABCDEF..."
if GOOGLE_API_KEY:
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY.strip()
print('Gemini KEY cargada?', bool(os.getenv('GOOGLE_API_KEY')))
print('Telegram TOKEN cargado?', bool(TELEGRAM_TOKEN))

Gemini KEY cargada? False
Telegram TOKEN cargado? True


In [4]:
import re
from typing import List, Dict
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Detectar si estamos en Colab para subir archivos
IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Gemini opcional
USE_GEMINI = False
GEM_MODEL = None
try:
    import google.generativeai as genai
    if os.getenv('GOOGLE_API_KEY'):
        genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
        GEM_MODEL = genai.GenerativeModel('gemini-1.5-flash')
        USE_GEMINI = True
        print('✅ Gemini listo.')
    else:
        print('ℹ️ Sin GOOGLE_API_KEY: usaré TF‑IDF.')
except Exception as e:
    print('ℹ️ Gemini no disponible (seguimos con TF‑IDF):', e)

# Estado global
KB_CHUNKS: List[Dict] = []
_VECTORIZER = None
_MATRIX = None

ℹ️ Sin GOOGLE_API_KEY: usaré TF‑IDF.


## 📥 3) Subir y leer archivos (usa **/content**)
### A) Subir desde el panel *Files*
En la izquierda, entra a **content** → icono de **subir** (flecha ↑) → elige tus archivos. Quedan en `/content`.

### B) O usar la función (abre diálogo): `subir_archivos()`


In [5]:

import os
os.environ["KAGGLE_USERNAME"] = "TU_USUARIO"
os.environ["KAGGLE_KEY"] = "TU_KEY"
!pip -q install kaggle
!kaggle datasets download -d zynicide/wine-reviews -p /content -o
!unzip -o /content/wine-reviews.zip -d /content

Dataset URL: https://www.kaggle.com/datasets/zynicide/wine-reviews
License(s): CC-BY-NC-SA-4.0
  0% 0.00/50.9M [00:00<?, ?B/s]
100% 50.9M/50.9M [00:00<00:00, 888MB/s]
Archive:  /content/wine-reviews.zip
  inflating: /content/winemag-data-130k-v2.csv  
  inflating: /content/winemag-data-130k-v2.json  
  inflating: /content/winemag-data_first150k.csv  


In [6]:
def subir_archivos() -> List[str]:
    """Diálogo para subir archivos a /content en Colab."""
    if not IN_COLAB:
        print('No estás en Colab. Copia tus archivos a /content manualmente.')
        return []
    up = files.upload()
    guardados = []
    for nombre, datos in up.items():
        with open(nombre, 'wb') as f:
            f.write(datos)
        guardados.append(nombre)
    print('✅ Subidos:', guardados)
    return guardados

def listar_utiles():
    import glob
    print('📄 Aquí hay:')
    print(glob.glob('*.pdf')+glob.glob('*.csv')+glob.glob('*.txt')+glob.glob('*.png')+glob.glob('*.jpg')+glob.glob('*.jpeg')+glob.glob('*.wav')+glob.glob('*.mp3'))

def extraer_texto_archivo(path: str) -> str:
    import os
    texto = ''
    ext = os.path.splitext(path)[1].lower()
    try:
        if ext == '.txt':
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                texto = f.read()
        elif ext == '.csv':
            tmp = pd.read_csv(path, encoding='utf-8', errors='ignore')
            texto = '\n'.join(tmp.astype(str).agg(' '.join, axis=1).tolist())
        elif ext == '.pdf':
            import pdfplumber
            with pdfplumber.open(path) as pdf:
                for page in pdf.pages:
                    texto += (page.extract_text() or '') + '\n'
        elif ext in ('.png', '.jpg', '.jpeg'):
            from PIL import Image
            import pytesseract
            texto = pytesseract.image_to_string(Image.open(path), lang='spa')
        elif ext == '.wav':
            import speech_recognition as sr
            r = sr.Recognizer()
            with sr.AudioFile(path) as source:
                audio = r.record(source)
            texto = r.recognize_google(audio, language='es-ES')
        elif ext == '.mp3':
            from pydub import AudioSegment
            import speech_recognition as sr
            wav = path + '.wav'
            AudioSegment.from_mp3(path).export(wav, format='wav')
            r = sr.Recognizer()
            with sr.AudioFile(wav) as source:
                audio = r.record(source)
            texto = r.recognize_google(audio, language='es-ES')
        else:
            print('Formato no soportado:', ext); return ''
    except Exception as e:
        print('Error leyendo', path, ':', e); return ''
    print('✅ Extraído', len(texto), 'caracteres de', path)
    return texto

def trocear_texto(texto: str, max_len: int = 900) -> List[str]:
    texto = re.sub(r'\s+', ' ', texto).strip()
    return [texto[i:i+max_len] for i in range(0, len(texto), max_len)]

def agregar_a_kb(path: str, texto: str, max_len: int = 900):
    global KB_CHUNKS
    for ch in trocear_texto(texto, max_len=max_len):
        if ch:
            KB_CHUNKS.append({'source': path, 'text': ch})
    print('KB ahora tiene', len(KB_CHUNKS), 'fragmentos.')

## 🧠 4) Entrenar índice y responder (Gemini opcional)
Si hay `GOOGLE_API_KEY`, Gemini redacta con el **contexto**; si no, devolvemos el mejor fragmento (TF‑IDF).

In [7]:
def entrenar_indice():
    global _VECTORIZER, _MATRIX
    if not KB_CHUNKS:
        print('Primero agrega contenido a la KB.'); return
    corpus = [c['text'] for c in KB_CHUNKS]
    _VECTORIZER = TfidfVectorizer(lowercase=True, ngram_range=(1,2))
    _MATRIX = _VECTORIZER.fit_transform(corpus)
    print('✅ Índice TF‑IDF entrenado con', len(corpus), 'fragmentos.')

def buscar_contexto(query: str, top_k: int = 4):
    if _VECTORIZER is None or _MATRIX is None:
        entrenar_indice()
    if _VECTORIZER is None or _MATRIX is None:
        return []
    vec = _VECTORIZER.transform([query])
    sims = cosine_similarity(vec, _MATRIX).ravel()
    idx = sims.argsort()[::-1][:top_k]
    return [KB_CHUNKS[i] | {'score': float(sims[i])} for i in idx]

def responder(query: str) -> str:
    ctx = buscar_contexto(query)
    if not ctx:
        return 'No tengo información aún. Sube archivos y entrena el índice.'
    if USE_GEMINI and GEM_MODEL:
        context_text = '\n\n'.join([c['text'] for c in ctx])
        prompt = (
            'Responde en español, breve y claro, usando SOLO el contexto.\n'
            "Si no está en el contexto, di: 'No tengo esa información aún'.\n\n"
            + 'Contexto:\n' + context_text + '\n\n'
            + 'Pregunta: ' + query + '\nRespuesta:'
        )
        try:
            resp = GEM_MODEL.generate_content(prompt)
            txt = getattr(resp, 'text', '')
            return txt.strip() or 'No tengo esa información aún.'
        except Exception as e:
            print('Gemini falló; uso TF‑IDF:', e)
    best = max(ctx, key=lambda x: x['score'])
    return best['text'][:500]

## ⚙️ 5) Demo rápida
1) (Colab) Ejecuta `subir_archivos()` o sube por el panel **Files**.
2) Procesa y agrega a la KB.  3) Entrena.  4) Pregunta.

In [14]:
# Subir (abre diálogo en Colab) y listar
archivos = subir_archivos() if IN_COLAB else []
listar_utiles()



Saving ESTUDIO_DE_SUELOS_SALON_COMUNAL_EL_TAMBO_NARINO.pdf to ESTUDIO_DE_SUELOS_SALON_COMUNAL_EL_TAMBO_NARINO.pdf
✅ Subidos: ['ESTUDIO_DE_SUELOS_SALON_COMUNAL_EL_TAMBO_NARINO.pdf']
📄 Aquí hay:
['ESTUDIO_DE_SUELOS_SALON_COMUNAL_EL_TAMBO_NARINO.pdf', 'winemag-data_first150k.csv', 'winemag-data-130k-v2.csv']


In [15]:
# Procesar lo subido y cargar a KB
for a in archivos:
    t = extraer_texto_archivo(a)
    if t:
        agregar_a_kb(a, t)
print('Fragmentos en KB:', len(KB_CHUNKS))

✅ Extraído 10662 caracteres de ESTUDIO_DE_SUELOS_SALON_COMUNAL_EL_TAMBO_NARINO.pdf
KB ahora tiene 12 fragmentos.
Fragmentos en KB: 12


In [16]:
entrenar_indice()
print(responder('¿Cuál es la actividad?'))

✅ Índice TF‑IDF entrenado con 12 fragmentos.
AL. En la Vereda la Ovejera, Con las estructuras que cumplan la norma NSR-10. DIEGO ROBLES BOLAÑOS ___________________________________________________________ INGENIERO CIVIL 2. OBJETIVOS 2. 1 Conocer el perfil estratigráfico del sub-suelo con base en el estudio de tres (3) apiques a cielo abierto. 2.2 Determinar mediante pruebas de campo y ensayos de laboratorio, las propiedades físicas mecánicas del suelo, tales como: Granulometría, límites de Atterberg, humedad natural, pesCI unitario y capac


## 💬 6) Telegram
Responde a **cualquier texto** que reciba usando `responder(query)`.
Antes de lanzar: asegura **KB > 0** y ejecutaste `entrenar_indice()`.

In [ ]:
import asyncio, nest_asyncio
from telegram.ext import Application, MessageHandler, filters

async def on_message(update, context):
    q = (update.message.text or '').strip()
    if not q:
        await update.message.reply_text('Envíame un texto y responderé con lo que haya en tus archivos.')
        return
    ans = responder(q)
    await update.message.reply_text(ans)

async def main_telegram():
    if not TELEGRAM_TOKEN:
        print('⚠️ Falta TELEGRAM_TOKEN. Pégalo en la celda de Claves.')
        return
    if not KB_CHUNKS or _VECTORIZER is None:
        print("⚠️ KB vacía o índice sin entrenar. Carga archivos y ejecuta 'entrenar_indice()' primero.")
    app = Application.builder().token(TELEGRAM_TOKEN).build()
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, on_message))
    print('🤖 Bot escuchando…')
    await app.run_polling()

nest_asyncio.apply()
# 👉 Descomenta para lanzar el bot:
await main_telegram()

🤖 Bot escuchando…


## 🆘 Ayuda rápida
- **¿Dónde subo?** A **/content** (panel Files → *content* → subir). Este cuaderno ya usa esa carpeta.
- **No encuentra archivo**: ejecuta la celda de ruta (muestra `/content`), vuelve a subir y usa el **nombre exacto**.
- **Gemini**: pega `GOOGLE_API_KEY` sin espacios extras y re‑ejecuta la celda de claves.
- **OCR/audio**: repite instalación si falla; usa imágenes claras o audio `.wav`.
- **Telegram**: pega `TELEGRAM_TOKEN`, asegúrate de KB>0 y de haber entrenado el índice.